In [1]:
import numpy as np
import tensorflow as tf

In [22]:
tf.reset_default_graph()
sentences = ["i love you","he loves me", "she likes baseball", "i hate you","sorry for that", "this is awful"]
labels = [1,1,1,0,0,0]
#做的是情感分类
wordlist = ' '.join(sentences).split()
wordlist = list(set(wordlist))
word_num = {w:i for i,w in enumerate(wordlist)}
vocab_size = len(word_num)

num_class = 2# 0表示好 1表示坏
sentence_size = 3#句子长度
embedding_size = 2
filter_sizes = [2,2,2]#卷积核的size
num_filter = 3#卷积核个数

input_x = []
for sen in sentences:
    input_x.append(np.asarray([word_num[word] for word in sen.split()]))
output = []
for label in labels:
    output.append(np.eye(num_class)[label])

X = tf.placeholder(tf.int32,[None,sentence_size])#batch_size sentences_size
Y = tf.placeholder(tf.int32,[None,num_class])#batch_size num_class

W = tf.Variable(tf.random_uniform([vocab_size,embedding_size],-1.0,1.0))
embed = tf.nn.embedding_lookup(W,X)#batch_size sentences_size embedding_size
embed = tf.expand_dims(embed,-1)#batch_size sentences_size embedding_size -1

pool_output = []
for i,filter_size in enumerate(filter_sizes):
    filter_shape = [filter_size,embedding_size,1,num_filter]#embed batch_size sentences_size embedding_size -1
    W = tf.Variable(tf.truncated_normal(filter_shape,stddev=0.1))
    b = tf.constant(0.1,shape=[num_filter])
    
    conv = tf.nn.conv2d(embed,#卷积
                        W,
                        strides = [1,1,1,1],
                        padding = 'VALID')
    h = tf.nn.relu(tf.nn.bias_add(conv,b))#非线性激活
    #max_pooling
    pool = tf.nn.max_pool(h,
                        ksize = [1,sentence_size-filter_size+1,1,1],# [batch_size, filter_height, filter_width, channel]
                        strides = [1,1,1,1],
                        padding = 'VALID')
    pool_output.append(pool)

filter_total = len(filter_sizes)*num_filter#卷积核个数*通道数
h_pool = tf.concat(pool_output,num_filter)# h_pool : [batch_size(=6), output_height(=1), output_width(=1), channel(=1) * 3]

h_pool = tf.reshape(h_pool,shape=[-1,filter_total])

Weights = tf.get_variable('W',shape=[filter_total,num_class],initializer=tf.contrib.layers.xavier_initializer())
bias = tf.Variable(tf.constant(0.1, shape=[num_class]))
model = tf.nn.xw_plus_b(h_pool,Weights,bias)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,logits=model))
optimizer = tf.train.AdamOptimizer(0.01).minimize(cost)

prediction = tf.nn.softmax(model)
prediction = tf.argmax(prediction,1)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(5000):
        _,loss = sess.run([optimizer,cost],feed_dict = {X:input_x,Y:output})
        if (i+1) % 1000 == 0:
            print('epoch:%d cost:%.6f' % ((i+1),loss))
            
    #test
    text = 'sorry hate you'
    test_num = [[word_num[w] for w in text.split()]]
    predict = sess.run([prediction],feed_dict={X:test_num})
    if predict[0][0] == 0:
        print(text,"is Bad Mean...")
    else:
        print(text,"is Good Mean!!")

epoch:1000 cost:0.000019
epoch:2000 cost:0.000004
epoch:3000 cost:0.000002
epoch:4000 cost:0.000001
epoch:5000 cost:0.000000
sorry hate you is Bad Mean...
